In [ ]:
from fastapi import FastAPI, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse
import tempfile
import os
from analysis import analyze
from report_writer import write_weekly_report
app = FastAPI()
# -----------------------------
# CORS
# -----------------------------
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)
# -----------------------------
# ANALYZE ONLY (JSON)
# -----------------------------
@app.post("/analyze")
async def analyze_endpoint(
    raw_file: UploadFile = File(...),
    progress_file: UploadFile = File(...),
    gms_file: UploadFile = File(...)
):
    with tempfile.TemporaryDirectory() as tmp:
        raw = os.path.join(tmp, "raw.xlsx")
        prog = os.path.join(tmp, "progress.xlsx")
        gms = os.path.join(tmp, "gms.csv")
        for f, p in [
            (raw_file, raw),
            (progress_file, prog),
            (gms_file, gms),
        ]:
            with open(p, "wb") as out:
                out.write(await f.read())
        return analyze(
            raw,
            prog,
            gms,
            progress_file.filename
        )
# -----------------------------
# ANALYZE + WORD REPORT
# -----------------------------
@app.post("/analyze-and-report")
async def analyze_and_report_endpoint(
    raw_file: UploadFile = File(...),
    progress_file: UploadFile = File(...),
    gms_file: UploadFile = File(...)
):
    with tempfile.TemporaryDirectory() as tmp:
        raw = os.path.join(tmp, "raw.xlsx")
        prog = os.path.join(tmp, "progress.xlsx")
        gms = os.path.join(tmp, "gms.csv")
        report_path = os.path.join(tmp, "Weekly_Top100_Report.docx")
        for f, p in [
            (raw_file, raw),
            (progress_file, prog),
            (gms_file, gms),
        ]:
            with open(p, "wb") as out:
                out.write(await f.read())
        # :small_blue_diamond: ANALYSIS (senin mevcut, dokunmadığımız kod)
        result = analyze(
            raw,
            prog,
            gms,
            progress_file.filename
        )
        # :small_blue_diamond: WORD RAPOR (senin verdiğin formatta)
        write_weekly_report(result, report_path)
        # :small_blue_diamond: DOWNLOAD RESPONSE
        return FileResponse(
            report_path,
            media_type="application/vnd.openxmlformats-officedocument.wordprocessingml.document",
            filename=f"Weekly_Top100_Report_Week_{result['week']}.docx"
        )